In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-shape-inversion/data/CRN_dataset.py
/kaggle/input/data-shape-inversion/data/synsetoffset2category.txt
/kaggle/input/data-shape-inversion/data/ply_dataset.py
/kaggle/input/data-shape-inversion/data/__init__.py
/kaggle/input/data-shape-inversion/data/CRN/our_data/test_data.h5
/kaggle/input/data-shape-inversion/data/CRN/our_data/mean_feature.h5
/kaggle/input/data-shape-inversion/data/CRN/our_data/valid_data.h5
/kaggle/input/data-shape-inversion/data/CRN/our_data/train_data.h5
/kaggle/input/data-shape-inversion/data/our_data/test_data.h5
/kaggle/input/data-shape-inversion/data/our_data/mean_feature.h5
/kaggle/input/data-shape-inversion/data/our_data/valid_data.h5
/kaggle/input/data-shape-inversion/data/our_data/train_data.h5
/kaggle/input/pretrained-models-shape-inversion/pretrained_models/car.pt
/kaggle/input/pretrained-models-shape-inversion/pretrained_models/.gitignore
/kaggle/input/pretrained-models-shape-inversion/pretrained_models/couch.pt
/kaggle/input/pretrained-

In [2]:
# ===================================================================
# CELL 1: FINAL SETUP (Corrects Chamfer3D Path)
# ===================================================================
import os
import shutil
import subprocess
import sys

# 1. Reset to Kaggle Working Root
os.chdir("/kaggle/working")
print(f"📍 Reset CWD to: {os.getcwd()}")

repo_url = "https://github.com/Vikky6789/shape_inversion_project_BTP.git"
repo_name = "shape_inversion_project_BTP"

# 2. Cleanup Old Repo
if os.path.exists(repo_name):
    print(f"🧹 Removing existing '{repo_name}'...")
    shutil.rmtree(repo_name)

# 3. Clone
print(f"⬇️ Cloning fresh copy...")
subprocess.check_call(f"git clone {repo_url}", shell=True)

# 4. Enter Repo
os.chdir(repo_name)
PROJECT_ROOT = os.getcwd()
sys.path.append(PROJECT_ROOT) # Critical for imports later
print(f"📍 Entered Project Root: {PROJECT_ROOT}")

# 5. Install Dependencies
print("\n📦 Installing ninja...")
subprocess.check_call("pip install -q ninja plyfile", shell=True)

# 6. Compile External Modules

# --- Compile Chamfer3D (The backend for ChamferDistancePytorch) ---
# FIX: Point to the 'chamfer3D' subfolder where setup.py lives
chamfer_src = os.path.join(PROJECT_ROOT, "external/ChamferDistancePytorch/chamfer3D")

if os.path.exists(chamfer_src):
    print(f"\n[1/2] Compiling Chamfer3D backend in: {chamfer_src}")
    os.chdir(chamfer_src)
    
    if os.path.exists("setup.py"):
        try:
            # We use install because it registers the package globally in the env
            subprocess.check_call("pip install .", shell=True)
            print("   ✅ Chamfer3D Compilation Successful")
        except subprocess.CalledProcessError as e:
            print(f"   ❌ Chamfer3D Compilation FAILED. Error: {e}")
    else:
        print(f"   ❌ ERROR: 'setup.py' still missing in {chamfer_src}")
else:
    print(f"\n❌ ERROR: Path not found: {chamfer_src}")

# --- Compile EMD ---
emd_path = os.path.join(PROJECT_ROOT, "external/emd")
if os.path.exists(emd_path):
    print(f"\n[2/2] Compiling EMD in: {emd_path}")
    os.chdir(emd_path)
    if os.path.exists("setup.py"):
        try:
            subprocess.check_call("pip install .", shell=True)
            print("   ✅ EMD Compilation Successful")
        except subprocess.CalledProcessError:
             print("   ❌ EMD Compilation FAILED.")
    else:
        print(f"   ❌ ERROR: 'setup.py' missing in {emd_path}")
else:
    print(f"\n❌ ERROR: Path not found: {emd_path}")

# 7. Final Return
os.chdir(PROJECT_ROOT)
print("\n✅ SETUP COMPLETE.")

📍 Reset CWD to: /kaggle/working
⬇️ Cloning fresh copy...


Cloning into 'shape_inversion_project_BTP'...
Updating files: 100% (529/529), done.


📍 Entered Project Root: /kaggle/working/shape_inversion_project_BTP

📦 Installing ninja...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.4 MB/s eta 0:00:00

[1/2] Compiling Chamfer3D backend in: /kaggle/working/shape_inversion_project_BTP/external/ChamferDistancePytorch/chamfer3D
Processing /kaggle/working/shape_inversion_project_BTP/external/ChamferDistancePytorch/chamfer3D
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chamfer_3D: filename=chamfer_3D-0.0.0-cp311-cp311-linux_x86_64.whl size=2701000 sha256=58cc0b4f8c9e5b83fdbe937403f05280f8c6a3eafae659a15359d58a63688e5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-i1vlqr_p/wheels/cd/a3/62/43f9a1e320bd0de46c387bf0ed7c3b86a12a841a5685e3f69d
Successfully built chamfer_3D
   ✅ Chamfer3D Compilation Successful

[2/2] Compiling EMD in: /kaggle/working/shape_inversion_project_BTP/external/emd
Processing /kaggle/working/shape_inversion_project_BTP/ex

In [3]:
# # Cell 2: Run the setup to compile EMD
# # We are already inside the 'shape_inversion_project_BTP' folder from the previous cell

# print("Starting Kaggle environment setup...")
# !pip install -q ninja

# # Navigate, compile, and return to the root
# %cd external/emd/
# !python setup.py install
# %cd /kaggle/working/shape_inversion_project_BTP/

# print("\nSUCCESS: Kaggle environment setup complete.")

In [4]:
import os
import shutil

# ================= CONFIGURATION =================
SOURCE_DIR = "/kaggle/input/pretrained-models-shape-inversion/pretrained_models"
DEST_DIR = "checkpoints" 
# =================================================

print(f"🛠️ Restructuring models from '{SOURCE_DIR}' to '{DEST_DIR}'...")

# 1. ROBUST CLEANUP (Fixes the OSError)
if os.path.exists(DEST_DIR):
    # Check if it is a Symbolic Link
    if os.path.islink(DEST_DIR):
        print(f"   🗑️ Removing existing symbolic link: {DEST_DIR}")
        os.unlink(DEST_DIR) # correctly removes the link
    else:
        print(f"   🗑️ Removing existing directory: {DEST_DIR}")
        shutil.rmtree(DEST_DIR) # correctly removes a real folder

# 2. Create fresh folder
os.makedirs(DEST_DIR, exist_ok=True)

# 3. Get list of all .pt files
model_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(".pt")]
print(f"   -> Found {len(model_files)} models.")

# 4. Restructure: Copy and Rename
for filename in model_files:
    # Example: "watercraft.pt" -> class "watercraft"
    class_name = filename.replace(".pt", "")
    
    # Create folder: checkpoints/watercraft/
    class_folder = os.path.join(DEST_DIR, class_name)
    os.makedirs(class_folder, exist_ok=True)
    
    src_path = os.path.join(SOURCE_DIR, filename)
    # Target: checkpoints/watercraft/checkpoint.pt
    dest_path = os.path.join(class_folder, "checkpoint.pt")
    
    shutil.copy(src_path, dest_path)
    print(f"   ✅ Created: {dest_path}")

print("\n🎉 DONE. Folder structure is now fixed!")

🛠️ Restructuring models from '/kaggle/input/pretrained-models-shape-inversion/pretrained_models' to 'checkpoints'...
   -> Found 8 models.
   ✅ Created: checkpoints/car/checkpoint.pt
   ✅ Created: checkpoints/couch/checkpoint.pt
   ✅ Created: checkpoints/lamp/checkpoint.pt
   ✅ Created: checkpoints/table/checkpoint.pt
   ✅ Created: checkpoints/cabinet/checkpoint.pt
   ✅ Created: checkpoints/watercraft/checkpoint.pt
   ✅ Created: checkpoints/chair/checkpoint.pt
   ✅ Created: checkpoints/plane/checkpoint.pt

🎉 DONE. Folder structure is now fixed!


In [5]:
# ===================================================================
# CELL 2: RUN EXPERIMENTS
# ===================================================================
import os
import subprocess
import sys

# --- 1. Setup ---
if os.path.basename(os.getcwd()) != "shape_inversion_project_BTP":
    if os.path.exists("shape_inversion_project_BTP"):
        os.chdir("shape_inversion_project_BTP")

PROJECT_ROOT = os.getcwd()
print(f"📍 Root: {PROJECT_ROOT}")

# --- 2. Configuration ---
script_name = "trainer.py"  # Using trainer.py as confirmed
ckpt_root = os.path.join(PROJECT_ROOT, "checkpoints")
KAGGLE_INPUT_ROOT = "/kaggle/input/data-shape-inversion"
ACTUAL_DATA_PATH = os.path.join(KAGGLE_INPUT_ROOT, "data/our_data") 
save_root = os.path.join(PROJECT_ROOT, "saved_results")

OBJECTS = ["watercraft"] 
LOSSES = ["cd", "dcd", "hypercd", "infocd", "uniformcd"]
SAVE_INTERVAL = 10

# --- 3. Environment ---
my_env = os.environ.copy()
# Add root to path so 'import external...' works
my_env["PYTHONPATH"] = f"{PROJECT_ROOT}:{KAGGLE_INPUT_ROOT}:{my_env.get('PYTHONPATH', '')}"

# --- 4. Execution Loop ---
for obj in OBJECTS:
    # Target: checkpoints/watercraft/checkpoint.pt
    ckpt_path = os.path.join(ckpt_root, obj, "checkpoint.pt")
    
    if not os.path.exists(ckpt_path):
        print(f"⚠️ [SKIPPING] {obj}: Checkpoint not found at {ckpt_path}")
        continue

    for loss in LOSSES:
        exp_name = f"CRN_{obj}_completion_{loss.upper()}"
        save_path = os.path.join(save_root, exp_name)
        os.makedirs(save_path, exist_ok=True)

        cmd = [
            "python", script_name,
            "--dataset", "CRN",
            "--class_choice", obj,
            "--inversion_mode", "completion",
            "--loss_func", loss,
            "--mapping_metric", loss,
            "--save_inversion_path", save_path,
            "--ckpt_load", ckpt_path,
            "--dataset_path", ACTUAL_DATA_PATH,
            "--save_interval", str(SAVE_INTERVAL),
            "--visualize"
        ]

        print(f"\n🚀 Running {obj} with {loss.upper()}...")
        
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=my_env)
        
        # Print output in real-time
        try:
            for line in proc.stdout:
                print(line, end="")
        except Exception as e:
            print(f"Stream error: {e}")
            
        proc.wait()
        
        if proc.returncode != 0:
            print(f"❌ Failed: {obj} - {loss}")
        else:
            print(f"✅ Finished: {obj} - {loss}")

print("\n✅ All Done.")

📍 Root: /kaggle/working/shape_inversion_project_BTP

🚀 Running watercraft with CD...
[INFO] Using device: cuda
Current Generator architecture:
Number of GCN layers: 7
Degrees: [1, 2, 2, 2, 2, 2, 64]
Features: [96, 256, 256, 256, 128, 128, 128, 3]

Layer 0 parameters:
  W_branch: torch.Size([1, 96, 96])
  bias: torch.Size([1, 1, 256])
  W_root.0.weight: torch.Size([256, 96])
  W_loop.0.weight: torch.Size([960, 96])
  W_loop.1.weight: torch.Size([256, 960])

Layer 1 parameters:
  W_branch: torch.Size([1, 256, 512])
  bias: torch.Size([1, 2, 256])
  W_root.0.weight: torch.Size([256, 96])
  W_root.1.weight: torch.Size([256, 256])
  W_loop.0.weight: torch.Size([2560, 256])
  W_loop.1.weight: torch.Size([256, 2560])

Layer 2 parameters:
  W_branch: torch.Size([2, 256, 512])
  bias: torch.Size([1, 2, 256])
  W_root.0.weight: torch.Size([256, 96])
  W_root.1.weight: torch.Size([256, 256])
  W_root.2.weight: torch.Size([256, 256])
  W_loop.0.weight: torch.Size([2560, 256])
  W_loop.1.weight: to